In [1]:
import math
import os
import random
import itertools

import torch
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

2023-07-15 21:34:16.941485: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# stdlib
import sys
import warnings

# synthcity absolute
from torch import mps
import synthcity.logger as log
from synthcity.plugins import Plugins
from synthcity.plugins.core.dataloader import TimeSeriesSurvivalDataLoader


log.add(sink=sys.stderr, level="INFO")
warnings.filterwarnings("ignore")

ImportError: Numba needs NumPy 1.24 or less

In [ ]:
import numpy as np
from synthcity.utils.datasets.time_series.pbc import PBCDataloader

(
    static_surv,
    temporal_surv,
    temporal_surv_horizons,
    outcome_surv,
) = PBCDataloader().load()
T, E = outcome_surv

horizons = [0.25, 0.5, 0.75]
time_horizons = np.quantile(T, horizons).tolist()

loader = TimeSeriesSurvivalDataLoader(
    temporal_data=temporal_surv,
    observation_times=temporal_surv_horizons,
    static_data=static_surv,
    T=T,
    E=E,
    time_horizons=time_horizons,
)

In [ ]:
df_synthcity = loader.dataframe()

In [ ]:
df_synthcity.head()

In [ ]:
# synthcity absolute
from synthcity.plugins import Plugins

Plugins(categories=["generic", "time_series"]).list()

In [ ]:
# synthcity absolute
from synthcity.plugins import Plugins

syn_model = Plugins().get("timegan")

syn_model.fit(loader)

In [ ]:
pbc2_data_generated = syn_model.generate(count=188).dataframe()

In [ ]:
pbc2_data_merged = pd.concat([df_synthcity, pbc2_data_generated], ignore_index=True)

In [ ]:
def load_pbc2():

  """Helper function to load and preprocess the SUPPORT dataset.
  The SUPPORT Dataset comes from the Vanderbilt University study
  to estimate survival for seriously ill hospitalized adults [1].
  Please refer to http://biostat.mc.vanderbilt.edu/wiki/Main/SupportDesc.
  for the original datasource.

  References
  ----------
  [1]: Knaus WA, Harrell FE, Lynn J et al. (1995): The SUPPORT prognostic
  model: Objective estimates of survival for seriously ill hospitalized
  adults. Annals of Internal Medicine 122:191-203.
  """

  # data = pkgutil.get_data(__name__, '')
  data = pbc2_data_merged

  drop_cols = ['seq_out_time_to_event', 'seq_out_event', 'seq_id', 'seq_time_id']

  outcomes = data.copy()
  outcomes['event'] =  data['seq_out_event']
  outcomes['time'] = data['seq_out_time_to_event']
  outcomes['seq_id'] = data['seq_id']
  outcomes['seq_time_id'] = data['seq_time_id']
  outcomes = outcomes[['event', 'time', 'seq_id', 'seq_time_id']]

  cat_feats = ["seq_static_sex", "seq_temporal_ascites", "seq_temporal_drug", "seq_temporal_edema", "seq_temporal_hepatomegaly", "seq_temporal_histologic", "seq_temporal_spiders"]
  num_feats = ["seq_temporal_SGOT", "seq_temporal_age", "seq_temporal_albumin", "seq_temporal_alkaline", "seq_temporal_platelets", "seq_temporal_prothrombin", "seq_temporal_serBilir", "seq_temporal_serChol"]

  return outcomes, data[cat_feats+num_feats]